In [75]:
import json
import requests

In [76]:
CREDENTIALS_FILE_PATH = "../spotify_key.config"
STORED_DATA_PATH = "../data/audio_features.json"

In [77]:
class DataRetriever:
    UNWANTED_AUDIO_FEATURES = ["id", "track_href", "analysis_url", "type"]

    def __init__(self):
        self.__auth_token = self.__get_auth_token()

    def __get_auth_token(self):
        with open(CREDENTIALS_FILE_PATH, "r") as input_file:
            credentials = input_file.read()
        client_id, client_secret = credentials.split("\n")

        response = requests.post(
            "https://accounts.spotify.com/api/token",
            {
                "grant_type": "client_credentials",
                "client_id": client_id,
                "client_secret": client_secret,
            },
        )

        auth_token = response.json()["access_token"]

        return auth_token

    def get_audio_features(self, track_id):
        response = requests.get(
            f"https://api.spotify.com/v1/audio-features/{track_id}",
            headers={"Authorization": f"Bearer {self.__auth_token}"},
        )

        audio_features = response.json()

        for audio_feature in self.UNWANTED_AUDIO_FEATURES:
            del audio_features[audio_feature]

        return audio_features
    


In [78]:
class DataSaver:
    @staticmethod
    def store(audio_features):
        with open(STORED_DATA_PATH, "w") as output_file:
            json.dump(audio_features, output_file, indent=4)

In [79]:
data_retriever = DataRetriever()
data = data_retriever.get_audio_features("23EOmJivOZ88WJPUbIPjh6")
DataSaver.store(data)